In [1]:

import pandas as pd
import csv
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import ast
import torch
import numpy as np
# from data_loader_asp import get_dataset, AspectDataset, AspectDatasetTrainer
# from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSeq2SeqLM, \
# Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, TrainingArguments, Trainer
# from torch.utils.data import DataLoader
# import evaluate
# from typing import Dict, List, Optional
# from dataclasses import dataclass, field
# from self_eval import calc_labs

# FILE_PATH = './restaurant_df.csv'
# dataset =  pd.read_csv(FILE_PATH).sample(frac=1).reset_index(drop=True)
# # asp_dset = AspectDataset(dataset,tokenizer)
# dataset = dataset.values
# val_split = 0.2

# INDEX_SPLIT = dataset.shape[0] - int(val_split * dataset.shape[0])
# TRAIN_DATASET = dataset[:INDEX_SPLIT, :]
# VALID_DATASET = dataset[INDEX_SPLIT:, :]

# datadict = {"train": TRAIN_DATASET, "validation": VALID_DATASET}

# tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base', max_model_lenght=512)

# model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')


/home/gauneg/anaconda3/envs/torch_test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dset = get_dataset(tokenizer, 'train')
validation_dset = get_dataset(tokenizer, 'validation')

model_dir = "./t5_trainer"

training_args = TrainingArguments(output_dir="test_trainer", 
optim='adafactor', 
num_train_epochs=4,
logging_strategy='steps',
learning_rate=3e-4,
logging_steps=100,
 save_strategy='epoch',
 per_device_train_batch_size=2,
 gradient_accumulation_steps=4
 )

In [4]:
from evaluate import load
exact_match_metric = load('exact_match')
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
      # print(inputs.keys())
      input_ids = inputs['input_ids']
      labels = inputs['labels'] 
      attention_mask = inputs['attention_mask']
      outputs = model(input_ids=input_ids, labels=labels, attention_mask=attention_mask)
      return (outputs["loss"], outputs["logits"]) if return_outputs else outputs["loss"]

# def compute_metrics(eval_pred):
#   logits, labels = eval_pred
#   predictions = torch.argmax(logits, axis=-1)
#   pred_str = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(predictions, skip_special_tokens=True))
#   true_str = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(labels, skip_special_tokens=True))
#   pred_arr = [word for word in pred_str.split(',') if word.lower()!='null']
#   true_arr = [word for word in true_arr.split(',') if word.lower()!='null']
#   return exact_match_metric(predictions=pred_arr, references=true_arr)
      

In [5]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dset,
    # eval_dataset=validation_dset,
    # compute_metrics=compute_metrics
)

In [6]:
trainer.train()

***** Running training *****
  Num examples = 1600
  Num Epochs = 8
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 1600
  Number of trainable parameters = 247577856


Step,Training Loss
100,2.002600
200,0.005600
300,0.003700
400,0.003800
500,0.002400
600,0.002200
700,0.001500
800,0.001500
900,0.001200
1000,0.001100


Saving model checkpoint to test_trainer/checkpoint-200
Configuration saved in test_trainer/checkpoint-200/config.json
Configuration saved in test_trainer/checkpoint-200/generation_config.json
Model weights saved in test_trainer/checkpoint-200/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-400
Configuration saved in test_trainer/checkpoint-400/config.json
Configuration saved in test_trainer/checkpoint-400/generation_config.json
Model weights saved in test_trainer/checkpoint-400/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-600
Configuration saved in test_trainer/checkpoint-600/config.json
Configuration saved in test_trainer/checkpoint-600/generation_config.json
Model weights saved in test_trainer/checkpoint-600/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-800
Configuration saved in test_trainer/checkpoint-800/config.json
Configuration saved in test_trainer/checkpoint-800/generation_config.json
Model weights saved in te

TrainOutput(global_step=1600, training_loss=0.12678282912587746, metrics={'train_runtime': 2598.5848, 'train_samples_per_second': 4.926, 'train_steps_per_second': 0.616, 'total_flos': 8764894111334400.0, 'train_loss': 0.12678282912587746, 'epoch': 8.0})

In [7]:
v_dload = DataLoader(validation_dset, batch_size=16)
from self_eval import calc_labs
fin_arr = []

In [8]:
for dpoint in v_dload:
    inp_ids = dpoint['input_ids']
    labels = dpoint['labels']
    amask = dpoint['attention_mask']
    y_pred = model.generate(inp_ids.to(model.device))
    for x, y, y_true in zip(inp_ids,y_pred, labels):
        pred = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y, skip_special_tokens=True))
        refs = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_true, skip_special_tokens=True))
        pred_arr = [y.strip().lower() for y in pred.split(',') if y.strip().lower()!='null']
        refs_arr = [y.strip().lower() for y in refs.split(',') if y.strip().lower()!='null']
        fin_arr.append([pred_arr, refs_arr])

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

/home/gauneg/anaconda3/envs/torch_test/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "p

In [9]:
calc_labs(fin_arr)


precision: 0.7381546134663342, recall: 0.7493670886075949, f1:0.7437185929648241


## Flan-T5 results without prompts
#### RESULTS WITH FLAN-T5-Small batch size 8, nsteps=800, gradient_accumulation_steps=1
`precision: 0.671604938271605, recall: 0.649164677804296, f1:0.6601941747572817`

#### RESULTS WITH FLAN-T5-BASE batch size 2, nsteps=800, gradient_accumulation_steps=4
`precision: 0.765, recall: 0.7409200968523002, f1:0.7527675276752769`

## FLAN-T5-BASE NO PROMPT
### MAMS 
`(0.7493806771263418, 0.8153638814016172, 0.7809810671256453)`
### Laptops 2014
`(0.3110938712179985, 0.39007782101167315, 0.3461372464393613)`
### Restaurant 2014
`0.591321243523316, 0.6880180859080633, 0.6360153256704981`

## FLAN-T5-BASE results with the following prompt 
find the aspect term in the text

### MAMS 
`0.7437268613739202, 0.8122192273135669, 0.7764655357526303`

### Laptops 2014
`0.25784615384615384, 0.4075875486381323, 0.31586882774217867`

### Restaurant 2014
`0.5540540540540541, 0.7106254709871892, 0.6226477385275668`

In [26]:
yx = model.generate(inputs=train_dset[1]['input_ids'].unsqueeze(dim=0), )


In [7]:
restaurant_train1 = '/home/gauneg/llm_experiments/ds_for_generation/restaurant_train_mams_gen.csv'
restaurant_train2 = '/home/gauneg/llm_experiments/ds_for_generation/dep_restaurant_rule_gen_14.csv'
df_train_restaurant1 = pd.read_csv(restaurant_train1)
df_train_restaurant2 = pd.read_csv(restaurant_train2)

In [8]:
combined_values = np.vstack((df_train_restaurant1.values, df_train_restaurant2.values))

In [10]:
np.random.shuffle(combined_values)

In [9]:
combined_values

array([['but the staff was so horrible to us.',
        "[('staff', None), ('staff', None)]"],
       ["to be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of teodora.",
        "[('teodora', None), ('food', None)]"],
       ["the food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.",
        "[('kitchen', None), ('menu', None), ('food', None)]"],
       ...,
       ['When we arrived at 6:00 PM, the restaurant was practically empty.',
        "[('restaurant', None)]"],
       ['Each table has a pot of boiling water sunken into its surface, and you get platters of thin sliced meats, various vegetables, and rice and glass noodles.',
        "[('glass noodles', None), ('various vegetables', None), ('rice', None), ('platters', None), ('thin sliced meats', None)]"],
       ['I am going to the mid town locatio

In [11]:
combined_values

array([['We went here for lunch a couple of weeks ago on a Saturday, and I was thoroughly impressed with the food.',
        "[('food', None)]"],
       ['my picks: Guizhou chicken, fish with hot bean source, fish fillet in spicy source (special menu).',
        "[('fish fillet', None), ('special menu', None), ('spicy source', None), ('bean source', None)]"],
       ['We walked in on a Wednesday night and were seated promptly.',
        '[]'],
       ...,
       ["In fact, you can't miss it.", "[('fact', None)]"],
       ['I have been about 4 times and have always had a great meal.',
        "[('meal', None)]"],
       ['Excellent experience.', "[('experience', None)]"]], dtype=object)

In [3]:
MAMS_PATH = '/home/gauneg/llm_experiments/mams_atsa_train.csv'
DEP_GENERATED_LAPTOP = '/home/gauneg/llm_experiments/ds_for_generation/dep_laptop_rule_gen_14.csv'
MAMS_GENRATED_LAPTOP = '/home/gauneg/llm_experiments/ds_for_generation/laptop_train_mams_gen.csv'

In [6]:
mams_df = pd.read_csv(MAMS_PATH)
dep_gen_laptop = pd.read_csv(DEP_GENERATED_LAPTOP)
mams_gen_laptop = pd.read_csv(MAMS_GENRATED_LAPTOP)

In [10]:
all_combined_laptop = np.vstack([mams_df.values, dep_gen_laptop.values, mams_gen_laptop.values])

In [11]:
all_combined_laptop.shape

(10393, 2)

In [15]:
ds_list = []
for text, lab_list in mams_df.values:
    fin_lab_arr = [(lab[0], None) for lab in ast.literal_eval(lab_list)]
    ds_list.append([text, fin_lab_arr])



In [19]:
df_out = pd.DataFrame(ds_list, columns=['text', 'labs'])

In [20]:
df_out.to_csv('./ds_for_generation/mams_train_in_gen_format.csv', index=None)